In [1]:
import requests
from bs4 import BeautifulSoup
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import time
import os

from sentence_transformers import SentenceTransformer

C:\Users\Sikma\AppData\Roaming\Python\Python312\site-packages\transformers\utils\hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


# Scrapping danych i zapis

## Utworzenie dataframe i słownika dat
utworzenie pustego dataframe politicians_df z potrzebnymi kolumnami i stworzenie słownika ze wszystkimi datami i posiedzeniami sejmu

In [2]:
politicians_df = pd.DataFrame(columns=["politician", "political_club", "utterance", "sejm_session", "date"])

In [3]:
gov_meetings_dict = {
    "2023-11-13": [1],
    "2023-11-14": [1],
    "2023-11-21": [1],
    "2023-11-22": [1],
    "2023-11-28": [1],
    "2023-11-29": [1],
    "2023-12-06": [1],
    "2023-12-07": [1],
    "2023-12-11": [1],
    "2023-12-12": [1],
    "2023-12-19": [1],
    "2023-12-20": [1],
    "2023-12-21": [1],
    "2024-01-16": [2],
    "2024-01-17": [2],
    "2024-01-18": [3],
    "2024-01-25": [4],
    "2024-01-26": [4],
    "2024-02-07": [5],
    "2024-02-08": [5],
    "2024-02-09": [5],
    "2024-02-21": [6],
    "2024-02-22": [6],
    "2024-03-06": [7],
    "2024-03-07": [7],
    "2024-03-08": [7],
    "2024-03-20": [8],
    "2024-03-21": [8],
    "2024-04-10": [9],
    "2024-04-11": [9],
    "2024-04-12": [9],
    "2024-04-24": [10],
    "2024-04-25": [10],
    "2024-04-26": [10],
    "2024-05-08": [11],
    "2024-05-09": [11],
    "2024-05-15": [11],
    "2024-05-22": [12],
    "2024-05-23": [12],
    "2024-06-12": [13],
    "2024-06-13": [13],
    "2024-06-14": [13],
    "2024-06-26": [14],
    "2024-06-27": [14],
    "2024-06-28": [14],
    "2024-07-11": [15],
    "2024-07-12": [15],
    "2024-07-23": [16],
    "2024-07-24": [16],
    "2024-07-25": [16],
    "2024-07-26": [16],
    "2024-09-11": [17],
    "2024-09-12": [17],
    "2024-09-13": [17],
    "2024-09-25": [18],
    "2024-09-26": [18],
    "2024-09-27": [18],
    "2024-10-01": [18],
    "2024-10-09": [19],
    "2024-10-10": [19],
    "2024-10-11": [19],
    "2024-10-16": [20],
    "2024-10-17": [20],
    "2024-10-18": [20],
    "2024-11-06": [21],
    "2024-11-07": [21],
    "2024-11-08": [21],
    "2024-11-19": [22],
    "2024-11-20": [22],
    "2024-11-21": [22],
    "2024-11-22": [22],
    "2024-11-27": [22],
    "2024-12-04": [23],
    "2024-12-05": [23],
    "2024-12-06": [24],
    "2024-12-18": [25],
    "2024-12-19": [25],
    "2024-12-20": [25],
    "2025-01-08": [26],
    "2025-01-09": [26],
    "2025-01-10": [26],
    "2025-01-22": [27],
    "2025-01-23": [27],
    "2025-01-24": [27]
}


## Scrapping danych przy użyciu sejmowego API
- przeiterowanie po każdej dacie i numerze posiedzenia sejmu,
- stworzenie URL do API sejmu zawierającego konkretną datę oraz posiedzenie i pobranie każdej wypowiedzi z danego zebrania
- wyjęcie z 'h2' polityka (politician)
- wyjęcie zmiędzy paragrafów 'p' wypowiedzi (utterance)
- zapisanie wszystkiego do politicians_df

In [4]:
for name, key in gov_meetings_dict.items():
    for number in key:
        i = 1
        start_time = time.time()
        while True:
            url = f"https://api.sejm.gov.pl/sejm/term10/proceedings/{number}/{name}/transcripts/{i}"
            page = requests.get(url)
            soup = BeautifulSoup(page.text, 'html')
        
            element = soup.find('h2', class_='mowca')
            if element is not None:
                politician = element.text.strip()[:-1]
            else:
                print(f'Koniec wypowiedzi. Odnotowano: {i} przemówień')
                break
        
            utterance_elements = []
            for p in soup.find_all('p')[1:]:
                utterance_elements.append(p.text.strip())
            utterance = "".join(utterance_elements).strip()
        
            nowy_wiersz = pd.DataFrame({"politician": [politician], 
                                        "utterance": [utterance], 
                                        "sejm_session": [number], 
                                        "date": [name]})
            politicians_df = pd.concat([politicians_df, nowy_wiersz], ignore_index=True)
        
            i = i+1
        end_time = time.time()
        print(f'Czas posiedzenia nr {number} o dacie {name}: {end_time-start_time:.2f} sekundy \n')

Koniec wypowiedzi. Odnotowano: 43 przemówień
Czas posiedzenia nr 1 o dacie 2023-11-13: 26.31 sekundy 

Koniec wypowiedzi. Odnotowano: 18 przemówień
Czas posiedzenia nr 1 o dacie 2023-11-14: 10.34 sekundy 

Koniec wypowiedzi. Odnotowano: 33 przemówień
Czas posiedzenia nr 1 o dacie 2023-11-21: 18.40 sekundy 

Koniec wypowiedzi. Odnotowano: 116 przemówień
Czas posiedzenia nr 1 o dacie 2023-11-22: 67.61 sekundy 

Koniec wypowiedzi. Odnotowano: 191 przemówień
Czas posiedzenia nr 1 o dacie 2023-11-28: 105.04 sekundy 

Koniec wypowiedzi. Odnotowano: 238 przemówień
Czas posiedzenia nr 1 o dacie 2023-11-29: 136.37 sekundy 

Koniec wypowiedzi. Odnotowano: 156 przemówień
Czas posiedzenia nr 1 o dacie 2023-12-06: 85.88 sekundy 

Koniec wypowiedzi. Odnotowano: 185 przemówień
Czas posiedzenia nr 1 o dacie 2023-12-07: 106.51 sekundy 

Koniec wypowiedzi. Odnotowano: 161 przemówień
Czas posiedzenia nr 1 o dacie 2023-12-11: 89.25 sekundy 

Koniec wypowiedzi. Odnotowano: 268 przemówień
Czas posiedzenia n

## Sprawdzenie poprawności dataframe
różne informacje: pierwsza wartość, próbka, shape itp.

In [5]:
politicians_df.iloc[0,0]

'Prezydent Rzeczypospolitej Polskiej Andrzej Duda'

In [6]:
politicians_df.sample().iloc[0,1]

'Szanowna Pani Marszałek! Szanowna Izbo! Zanim zostałem posłem, byłem radnym \r\nmiasta Częstochowy. To jest pierwsze miasto w Polsce, które wprowadziło in \r\nvitro w 2012 r. W 2015 r. był taki moment, że radni Częstochowy chcieli \r\nzaktualizować ten program, zwiększyć kwotę przyznawaną parom i klub Lewicy i \r\nPlatformy był za, a klub PiS-u był przeciw.W 2015 r. to głosy naszego częstochowskiego klubu, klubu mieszkańców \r\nCzęstochowy, zdecydowały o tym, że do tej pory w Częstochowie funkcjonuje in \r\nvitro i urodziło się 77 dzieci.Cieszę się, że historia zatacza koło, bo wtedy w Częstochowie mieliśmy duży \r\ndylemat przez to, że mieszkańcy, osoby zameldowane w Częstochowie mogą \r\nkorzystać z tego programu, a mieszkańcy sąsiednich gmin (Dzwonek) jak \r\nBlachownia, Koniecpol czy Lubliniec - nie. Cieszę się, że mogę to dzisiaj \r\nnaprawić i razem z państwem zagłosować za in vitro. Dziękuję bardzo. \r\n(Oklaski)'

In [8]:
politicians_df.shape

(15878, 4)

In [9]:
politicians_df.groupby('politician').size().sort_values(ascending=False)

politician
Poseł Witold Tumanowicz                                                                                                       360
Poseł Jarosław Sachajko                                                                                                       322
Poseł Zbigniew Bogucki                                                                                                        205
Poseł Tadeusz Tomaszewski                                                                                                     199
Poseł Grzegorz Braun                                                                                                          182
                                                                                                                             ... 
Poseł Zbigniew Chmielowiec (tekst niewygłoszony)                                                                                1
Poseł Sprawozdawca Alicja Chybicka                                             

## Dodanie danych o partiach politycznych
wykorzystanie sejmowego API do stworzenia słownika zawierającego partie polityczne i ich posłów, a następnie przeiterowanie po datasecie politicians_df aby nadać politykom ich kluby

In [2]:
url_politicians = "https://api.sejm.gov.pl/sejm/term10/MP"
page_politicians = requests.get(url_politicians)
political_parties_data = page_politicians.json()

In [12]:
club_dict = {}

for person in political_parties_data:
    club = person.get("club", "Brak klubu")
    name = person.get("firstLastName", "Nieznany")

    # Dodajemy do słownika
    if club in club_dict:
        club_dict[club].append(name)
    else:
        club_dict[club] = [name]

for club, names in club_dict.items():
    print(f"{club}: {', '.join(names)}\n")

PiS: Andrzej Adamczyk, Adam Andruszkiewicz, Waldemar Andzel, Dorota Arciszewska-Mielewczyk, Iwona Ewa Arent, Marek Ast, Piotr Babinetz, Ryszard Bartosik, Barbara Bartuś, Mariusz Błaszczak, Rafał Bochenek, Jacek Bogucki, Zbigniew Bogucki, Joanna Borowiak, Kamil Bortniczuk, Bożena Borys-Szopa, Waldemar Buda, Lidia Burzyńska, Zbigniew Chmielowiec, Artur Chojecki, Kazimierz Bogusław Choma, Dominika Chorosińska, Tadeusz Chrzan, Anna Ewa Cicholska, Krzysztof Ciecióra, Janusz Cieszyński, Michał Cieślak, Krzysztof Czarnecki, Witold Wojciech Czarnecki, Przemysław Czarnek, Arkadiusz Czartoryski, Anita Czerwińska, Katarzyna Czochara, Władysław Dajczak, Anna Dąbrowska-Banaszek, Zbigniew Dolata, Bartłomiej Dorywalski, Przemysław Drabek, Elżbieta Duda, Michał Paweł Dworczyk, Jan Michał Dziedziczak, Magdalena Filipek-Sobczak, Radosław Fogiel, Andrzej Gawron, Grzegorz Gaża, Anna Gembicka, Szymon Giżyński, Piotr Gliński, Małgorzata Golińska, Kazimierz Gołojuch, Robert Gontarz, Mariusz Gosek, Małgorzata

In [13]:
politicians_df["political_club"] = ""

In [22]:
club_dict = {}

def assign_political_club(row):
    politician_name = row["politician"]

    for club, names in club_dict.items():
        if any(name in politician_name for name in names):  # Szukamy nazwiska w całym stringu
            return club
    return "Nieznana partia"

politicians_df["political_club"] = politicians_df.apply(assign_political_club, axis=1)

In [35]:
politicians_df.sample()

,politician,utterance,sejm_session,date,political_club
8799,Poseł Adam Krzemiński,Panie Marszałku! Wysoka Izbo! Szanowna Pani Mi...,17,2024-09-12,KO


## Zapis dataframe do excela
plik z rozszerzeniem .xlsx

In [30]:
politicians_df.to_excel('politycy.xlsx', index=False)

# Operowanie na zescrappowanych danych

## Przygotowanie datasetu z excela
- Załadowanie datasetu z wypowiedziami polityków z rozszerzeniem .csv
- Oczyszczenie datasetu (wypowiedzi - utterance) z niepotrzebnych afixów "\r" "\n"

In [16]:
dataset_politycy = pd.read_csv('politycy.csv', encoding='utf-8', sep=';')
dataset_politycy = dataset_politycy.dropna(subset=["utterance", "politician", "sejm_session", "date", "political_club"])

In [17]:
dataset_politycy['utterance'] = dataset_politycy['utterance'].str.replace(r'[\n\r]', '', regex=True)

In [29]:
dataset_politycy['utterance'][9003]

'Panie Marszałku! Wysoka Izbo! Oczywiście jest to takie marzenie, że samorząd to jest taka organizacja, na której czele stoi gospodarz wybrany w jednomandatowym okręgu wyborczym...(Głos z sali: Bo tak jest. Tak jest.)...gospodarz, który poza światem własnej gminy, własnego powiatu czy miasta nie widzi innych trosk politycznych. Wyciągnięcie samorządów z polityki jest celem samym w sobie. I tak sobie wyobrażam, że w tym marzeniu, w którym burmistrzowie i prezydenci zajmują się jedynie sprawami swoich mieszkańców, jest taki moment, kiedy razem ze swoją radą siada się do stołu i tworzy się budżet. Ta ustawa, wydaje się, idzie w dobrym kierunku, bo nie ma tam już punktu: ile dadzą albo nie dadzą. Budżet miasta, gminy, wsi musi opierać się na jakichś pewnikach, czyli mamy własne dochody. Ja tutaj jako przedsiębiorca widzę bardzo duży pozytyw, jeśli chodzi o przerzucenie dochodów właśnie związanych z CIT, PIT, dlatego że miasta być może zaczną wtedy dbać o własnych przedsiębiorców, zaczną wi

In [31]:
dataset_politycy.to_excel('politycy_czysty.xlsx', index=False)

## Podzielenie wypowiedzi na mniejsze segmenty (niekonieczne)
- Wykonanie na każdym wierszu funkcji podzielenia i pofiltrowania
- Podział wypowiedzi na zdania, punkty graniczne to między innymi kropka, przecinek, znak zapytania itp.
- Pofiltrowanie aby w datasecie zostały TYLKO zdania zawierające więcej niż 10 wyrazów

In [10]:
# poucinanie wypowiedzi na pojedyncze zdania i odrzucenie tych które są krótsze niż 15 słów

import re

def split_and_filter(row):
    #sentences = re.split(r'(?<=[.!?])\s+', row['utterance'])
    sentences = re.split(r'(?<=[.!?])\s*(?=[A-ZĄĆĘŁŃÓŚŹŻ])', row['utterance'])
    filtered_sentences = [s for s in sentences if len(s.split()) >= 15]
    return pd.DataFrame({'politician': row['politician'], 'utterance': filtered_sentences, 'political_club': row['political_club']})

dataset_filtered_politycy = pd.concat(dataset_politycy.apply(split_and_filter, axis=1).tolist(), ignore_index=True)
dataset_filtered_politycy.to_excel('politycy_filtered.xlsx', index=False)

# Model AI oraz embeddings

## Przygotowanie datasetu z excela
Załadowanie datasetu z wypowiedziami polityków z rozszerzeniem .csv

In [32]:
#dataset_filtered_politycy = pd.read_csv('politycy_filtered.csv', encoding='utf-8', sep=';')
#dataset_filtered_politycy = dataset_filtered_politycy.dropna(subset=["politician", "utterance", 'political_club'])

dataset_filtered_politycy = pd.read_csv('politycy_czysty.csv', encoding='utf-8', sep=';')
dataset_filtered_politycy = dataset_filtered_politycy.dropna(subset=["politician", "utterance", 'political_club'])

## Załadowanie odpowiedniego modelu AI oraz stworzenie funkcji
- funkcja get_embedding odpowiedzialna za stworzenie embeddingu na tekście
- funkcja get_similarity porównująca podobieństwo między embeddingami przy użyciu cosine similarity 

In [9]:
#model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
#model = SentenceTransformer('BAAI/bge-multilingual-gemma2')
#model = SentenceTransformer('Alibaba-NLP/gte-multilingual-base')
#model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
#model = SentenceTransformer('sentence-transformers/stsb-xlm-r-multilingual')

In [22]:
# Funkcja do konwersji tekstu na wektor (embedding)
def get_embedding(text):
    return model.encode(text)


def get_similarity(user_input, df_embeddings):
    user_embedding = get_embedding(user_input)
    
    similarities = []
    for index, row in df_embeddings.iterrows():
        politician_embedding = np.array(row['embedding'])
        
        similarity = cosine_similarity([user_embedding], [politician_embedding])[0][0]
        similarities.append((similarity, row['politician'], row['utterance']))

    similarities.sort(reverse=True, key=lambda x: x[0])
    top_n = 4
    avg_similarity = np.mean([sim for sim, _, _ in similarities[:top_n]])
    
    return avg_similarity, similarities[:top_n]


def get_similarity_politician(user_input, df_dataset):
    user_embedding = get_embedding(user_input)
    
    df_dataset["similarity"] = df_dataset["embedding"].apply(lambda emb: cosine_similarity([user_embedding], [emb])[0][0])
    df_sorted = df_dataset.sort_values(by="similarity", ascending=False)

    top_n = 3
    top_similar = df_sorted.head(top_n)

    return top_similar[["similarity", "politician", "political_club", "utterance"]].values.tolist()

## Sprawdzenie liczby tokenów dla każdej z wypowiedzi sejmowych

In [36]:
from transformers import AutoModel
model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
model = AutoModel.from_pretrained(model_name)
print(model.config)
print(model.config.max_position_embeddings)

BertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 250037
}

512


In [10]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
print(model)
print("max_seq_length:", model.max_seq_length)
model.max_seq_length = 512
print("max_seq_length:", model.max_seq_length)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)
max_seq_length: 128
max_seq_length: 512


In [41]:
def count_tokens_in_csv(df):
    tokenizer = model.tokenizer 

    df["token_count"] = df["utterance"].astype(str).apply(
        lambda x: len(tokenizer.encode(x, add_special_tokens=True))
    )
    
    return df

In [43]:
df_with_tokens = count_tokens_in_csv(dataset_filtered_politycy)
print(df_with_tokens.head())

                                         politician  \
0  Prezydent Rzeczypospolitej Polskiej Andrzej Duda   
1                 Sekretarz Poseł Bożena Żelazowska   
2            Sekretarz Poseł Alicja Łepkowska-Gołaś   
3                 Sekretarz Poseł Bożena Żelazowska   
4            Sekretarz Poseł Alicja Łepkowska-Gołaś   

                                           utterance  sejm_session  \
0  Drodzy Rodacy! Wielce Szanowny Panie Marszałku...             1   
1  (Sekretarz poseł odczytuje nazwiska posłów)Pos...             1   
2  Poseł Andrzej Adamczyk: Ślubuję. Tak mi dopomó...             1   
3  Poseł Agnieszka Pomaska: Ślubuję.Poseł Cezary ...             1   
4  Poseł Mirosław Suchoń: Ślubuję. Tak mi dopomóż...             1   

         date   political_club  token_count  
0  2023-11-13  Nieznana partia         5002  
1  2023-11-13           PSL-TD          376  
2  2023-11-13               KO          361  
3  2023-11-13           PSL-TD          300  
4  2023-11-13     

In [46]:
df_with_tokens[df_with_tokens["token_count"] < 1000].sort_values("token_count")

,politician,utterance,sejm_session,date,political_club,token_count
6952,Członek Państwowej Komisji do spraw Przeciwdzi...,Ślubuję. (Oklaski),13,2024-06-14,Nieznana partia,11
24,Sekretarz Poseł Alicja Łepkowska-Gołaś,Poseł Bożena Żelazowska: Ślubuję. Tak mi dopom...,1,2023-11-13,KO,23
14027,Sekretarz Poseł Małgorzata Gromadzka,"Informuję, że dziś odbędzie się posiedzenie Ko...",24,2024-12-06,KO,23
25,Sekretarz Poseł Bożena Żelazowska,Poseł Alicja Łepkowska-Gołaś: Ślubuję. (Oklaski),1,2023-11-13,PSL-TD,24
7697,Poseł Piotr Paweł Strach,W imieniu wnioskodawców chciałbym wycofać 6. w...,15,2024-07-12,Polska2050-TD,24
...,...,...,...,...,...,...
2532,Sekretarz Stanu w Ministerstwie Rozwoju i Tech...,Szanowny Panie Marszałku! Wysoka Izbo! Panie p...,4,2024-01-26,KO,997
2670,Poseł Jarosław Sachajko,Panie Marszałku najbardziej na świecie otwarte...,5,2024-02-07,Republikanie,997
3197,Poseł Joanna Wicha,Szanowna Pani Marszałek! Pani Minister! Wysoka...,6,2024-02-21,Lewica,998
3559,Sekretarz Stanu w Ministerstwie Aktywów Państw...,Pani Marszałek! Wysoka Izbo! Bardzo dziękuję w...,7,2024-03-06,KO,998


In [47]:
politycy_phrases_embeddings = df_with_tokens[df_with_tokens["token_count"] < 1000]
print(len(politycy_phrases_embeddings))
print(politycy_phrases_embeddings.sample())

13304
                            politician  \
2725  Poseł Zbigniew Krzysztof Kuźmiuk   

                                              utterance  sejm_session  \
2725  Pani Marszałek! Pani Minister! Wysoka Izbo! Pa...             5   

            date political_club  token_count  
2725  2024-02-07            PiS          348  


## Konwersja dataframe'u z wypowiedziami na embeddings
wykonanie "get_embedding" na każdej wypowiedzi z dataframe

In [48]:
print("max_seq_length:", model.max_seq_length)

max_seq_length: 512


In [49]:
from tqdm import tqdm

politycy_phrases_embeddings = [
    (get_embedding(row["utterance"]), row["politician"], row["utterance"], row['political_club']) 
    for _, row in tqdm(politycy_phrases_embeddings.iterrows(), total=len(politycy_phrases_embeddings))
]

df_politycy_phrases_embeddings = pd.DataFrame(politycy_phrases_embeddings, columns=["embedding", "politician", "utterance", 'political_club'])
print(df_politycy_phrases_embeddings.shape)

data = []
for idx, row in df_politycy_phrases_embeddings.iterrows():
    embedding = row['embedding']  # Dostęp do kolumny 'embedding'
    politician = row['politician']
    utterance = row['utterance']
    political_club = row['political_club']
    
    data.append({
        "embedding": ','.join(map(str, embedding)),  # Zapis embeddingu jako string
        "politician": politician,
        "utterance": utterance,
        "political_club": political_club
    })

df_politycy_phrases_embeddings = pd.DataFrame(data)
#df_politycy_phrases_embeddings.to_excel('politycy_embeddings_stsb.xlsx', index=False)
df_politycy_phrases_embeddings.to_excel('politycy_embeddings_minilm12.xlsx', index=False)

100%|██████████| 13304/13304 [25:47<00:00,  8.60it/s] 


(13304, 4)


In [ ]:
len(df_politycy_phrases_embeddings)

In [22]:
df_politycy_phrases_embeddings.head(5)

,embedding,politician,utterance,political_club
0,"[-0.13537993, 0.3929176, 0.02940008, -0.283716...",Prezydent Rzeczypospolitej Polskiej Andrzej Duda,"Ekscelencjo, Czcigodny Księże Arcybiskupie, Dz...",Nieznana partia
1,"[0.06613037, 0.33783805, 0.32265285, 0.0248571...",Prezydent Rzeczypospolitej Polskiej Andrzej Duda,"Dziękuję, dziękuję i jeszcze raz dziękuję wszy...",Nieznana partia
2,"[-0.010987518, 0.08707595, -0.072819464, 0.147...",Prezydent Rzeczypospolitej Polskiej Andrzej Duda,"Oczywiście my, Polacy, w wielu sprawach istotn...",Nieznana partia
3,"[-0.13236119, 0.27273113, -0.3124308, -0.14640...",Prezydent Rzeczypospolitej Polskiej Andrzej Duda,"Najważniejsze jednak jest to, że spory te rozs...",Nieznana partia
4,"[-0.34609145, 0.33584738, -0.019541811, -0.048...",Prezydent Rzeczypospolitej Polskiej Andrzej Duda,"Polacy ponownie uwierzyli, że ich głos ma znac...",Nieznana partia


# Porównywanie wypowiedzi użytkownika z politykami

## Przygotowanie datasetu z excela
Załadowanie datasetu z wypowiedziami polityków z rozszerzeniem .csv oraz dokonanie konwersji na obiekt typu array na kolumnie "embedding"

In [3]:
df_politycy_phrases_embeddings = pd.read_csv('politycy_embeddings_minilm12.csv', encoding='utf-8', sep=';')
df_politycy_phrases_embeddings = df_politycy_phrases_embeddings.dropna(subset=["embedding", "politician", "utterance", 'political_club'])

In [6]:
df_politycy_phrases_embeddings.sample()

,embedding,politician,utterance,political_club
11561,"[0.22658972, 0.12388311, 0.08822335, -0.045189...",Poseł Patryk Wicher,Szanowna Pani Marszałek! Wysoka Izbo! Szanowni...,PiS


In [5]:
df_politycy_phrases_embeddings["embedding"] = df_politycy_phrases_embeddings["embedding"].apply(lambda x: np.array(list(map(float, x.split(',')))))

In [7]:
print(f"Liczba wierszy: {len(df_politycy_phrases_embeddings)}")
print(f"Kształt DataFrame: {df_politycy_phrases_embeddings.shape}")
display(df_politycy_phrases_embeddings.sample())

Liczba wierszy: 13304
Kształt DataFrame: (13304, 4)


,embedding,politician,utterance,political_club
6417,"[0.06982937, 0.08433071, -0.23477678, 0.168101...",Poseł Elżbieta Duda,"Szanowna pani marszałek, Wysoka Izbo, historia...",PiS


## Porównywanie

In [23]:
user_statement = "Zielona energia jest bardzo istotna i powinniśmy skupić się na rozwoju rozwiązaniach ekologicznych. Musimy redukować energię węglową i zastapić ją energią z wiatru czy z wody."
#user_input = "Polacy potrzebują legalnej aborcji, ponieważ to jest prawo kobiety i nie można przymuszać kobiet."
#user_input = "Aborcja to zabójstwo dziecka i nie można tego zalegalizować. Jest to sprzeczne z jakimikolwiek prawami moralnymi i religijnymi"
#user_input = "Aborcja powinna być dozwolona w przypadku gwaltu, ciężkiej choroby dziecka lub możliwej śmierci matki. Ale nigdy nie dozwolona na życzenie."
#user_input = "Prawo do posiadania broni powinno być dozwolone dla przeciętnego obywatela. Ludzie muszą mieć możliwość obrony wlasnej."
#user_input = "Polska powinna wspierać Ukrainę w tej wojnie i zagwarantować jej wszelką pomoc finansową i militarną."

#avg_similarity, top_similarities = get_similarity(user_input, df_politycy_phrases_embeddings)
similiar_results = get_similarity_politician(user_statement, df_politycy_phrases_embeddings)

print(user_statement)
for similarity, politician, political_club, utterance in similiar_results:
    print(f"Polityk: {politician}, Partia: {political_club}, Podobieństwo: {similarity}, \n Wypowiedź: {utterance} \n")

Zielona energia jest bardzo istotna i powinniśmy skupić się na rozwoju rozwiązaniach ekologicznych. Musimy redukować energię węglową i zastapić ją energią z wiatru czy z wody.
Polityk: Poseł Marcin Skonieczka, Partia: Polska2050-TD, Podobieństwo: 0.6070108426793767, 
 Wypowiedź: Panie Marszałku! Wysoka Izbo! 10 stycznia obchodzimy Dzień Obniżania Kosztów Energii. Inicjatywa ta została zapoczątkowana w 1995 r. przez organizację z Kanady. Celem tej inicjatywy jest promowanie działań mających na celu redukcję zużycia energii, co pozytywnie wpływa na ochronę środowiska i jednocześnie przekłada się na oszczędności finansowe. Oszczędzanie energii nie tylko odciąża domowe budżety, lecz również przyczynia się do redukcji emisji CO2, wspierając walkę ze zmianami klimatycznymi.W Dniu Obniżania Kosztów Energii zachęcam Polki i Polaków do zapoznania się z opublikowanym na stronach rządowych opracowaniem ˝Dobre praktyki w zakresie zmniejszenia zużycia energii˝. Niech Dzień Obniżania Kosztów Energii